# Alive & Thrive
## Viet Nam Data: Women Files - Preprocessing
## Prepared by Aaron Wise; aaron@a3di.dev
### Version: 25 May 2022

In [1]:
# %load std_imports
from pathlib import Path

import numpy as np
import pandas as pd

import json
import pyreadstat

from std_utils import (
    read_file,
    run_quality_assurance,
    generate_HHID,
    merge_hh_hl_data,
    subset_df,
    standardize_col_names,
    standardize_col_values,
    save_merge,
)

pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", None)


In [2]:
# %load women_imports.py
from pathlib import Path

import numpy as np
import pandas as pd

import json
import pyreadstat

from women_analysis import create_anc_4_visits, create_anc_3_components

from aw_analytics import mean_wt, output_mean_table

pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", None)


In [3]:
# Set parameters
country = 'VNM'
recode = 'women'

# -------------------------------------------------------------------
year = '2021'
# -------------------------------------------------------------------


In [4]:
# Read file
df = read_file(country, year, recode)

# Create HHID to facilitate merge of HH and HL data
generate_HHID(df, country, year, recode)

# Run quality assurance
run_quality_assurance(df)

# Merge in HH and HL data
df = merge_hh_hl_data(df, country, year)

The file -- wm_2021.sav -- has the following shape: Rows: 11294; Columns: 458
HHID is NOT unique
Drop columns if all values are NaN...
Updated -- Rows: 11294; Columns: 450
Checking if any rows are duplicates...
The are no duplicate rows


In [5]:
# Select completed questionnaires (WM17) and birth in past two years (CM17)
print(df.shape)
df = df[(df['WM17'] == "COMPLETED") & (df['CM17'] == "ONE OR MORE LIVE BIRTHS IN THE LAST 2 YEARS")]
print(df.shape)

(11294, 464)
(1566, 464)


#### Create Outcome Indicators

In [6]:
# --- ANC 4+ visits [anc_4_visits] --- #
create_anc_4_visits(df, country, year)

In [7]:
# --- ANC components [anc_3_components] --- #

create_anc_3_components(df, country, year)

In [ ]:
# --- Institutional [inst_delivery] --- #

## Create categories
inst_delivery_options = ['PUBLIC HOSPITAL', 'PRIVATE HOSPITAL', 'COMMUNE HEALTH CENTRE',
 'LOCAL CLINIC', 'HOSPITAL OF A MINISTRY OR A SECTOR', 'OTHER PRIVATE MEDICAL']

# Create indicator
df['inst_delivery'] = np.where(df['MN20'].isin(inst_delivery_options), 100, 0)

In [10]:
var = 'anc_3_components'
ind_vars = ['HH6']
wt = 'wmweight'

output_mean_table(df, var, ind_vars, wt)

,anc_3_components,Weighted_Count
HH6,,
RURAL,68.8,987.2
URBAN,85.6,449.2


In [ ]:
# --- Caesarean Delivery [caesarean_del] --- #

# Create indicator
df['caesarean_del'] = np.where(df['MN21'] == 'YES', 100, 0)

In [ ]:
# --- Post-natal Health Check (mother) [pnc_mother] --- #

## Health check by health provider after birth & a) before leaving facility (PN5) or b) before health provider left home (PN9)

## Create sub-indicator
df['health_check_after_birth'] = np.where(((df['PN5'] == 'YES') | (df['PN9'] == 'YES')), 100, 0)

## Seprate PNC visit by health provider (PN22U and PN22N)

## Post-natal care visit within 2 days ##

### Convert PN22N to float
df['PN21N'] = np.where(df['PN22N'] == 'DK', '99.0', df['PN22N'])
df['PN22N'] = df['PN22N'].astype(float)

## Create sub-indicator
df['pnc_2_days'] = np.where((df['PN22U'] == 'Hours') | ((df['PN22U'] == 'Days') & (df['PN22N'] <= 2)), 100, 0)

### Set value pnc_2_days to 0 IF check was not done by health provider
df['pnc_health_provider'] = np.where((df['PN23A'] == 'DOCTOR') | (df['PN23B'] == 'NURSE / MIDWIFE') |\
     (df['PN23F'] == 'TRADITIONAL BIRTH ATTENDANT') | (df['PN23G'] == 'COMMUNITY HEALTH WORKER'), 100, 0)

df['pnc_2_days'] = np.where((df['pnc_health_provider'] == 100), df['pnc_2_days'], 0)

# Create indicator
df['pnc_mother'] = np.where((df['health_check_after_birth'] == 100) | (df['pnc_2_days'] == 100), 100, 0)

In [ ]:
# --- Low birthweight [low_bw] --- #

## Convert categorical column to int

### Convert DK and NaN to 99
df['MN34'] = np.where(df['MN34'] == 'DK', 'NaN', df['MN34'])

### Cast value to float 
df['MN34'] = df['MN34'].astype(float)

# Create indicator
df['low_bw'] = np.where(df['MN34'] < 2.5, 100, 0)
df['low_bw'] = np.where(df['MN34'].isnull(), np.nan, df['low_bw'])

In [ ]:
# --- Early Initiation BF [early_bf] --- #
## Convert categorical column to int

### Convert DK to 0
df['MN37N'] = np.where(df['MN37N'] == 'IMMEDIATELY', '0.0', df['MN37N'])
df['MN37N'] = np.where(df['MN37N'] == 'DK / DON’T REMEMBER', '99.0', df['MN37N'])

### Cast value to float 
df['MN37N'] = df['MN37N'].astype(float)

# Create indicator
df['early_bf'] = np.where(df['MN37N'] < 1, 100, 0)

In [ ]:
# Weighted mean
wt = ['wmweight']
var = 'early_bf'
ind_vars = ['HH7']

output_mean_table(df, var, ind_vars, wt)

#### Create Equity Stratifiers

In [ ]:
# --- Residence --- #
df['residence'] = np.where(df['HH6'] == 'RURAL', 'Rural', 'Urban')

# --- Location --- #

## Combine region value with city value
df['region_comb'] = df['HH7']
df['region_comb'] = np.where(df['HH7A'] == 'Ha Noi', 'Ha Noi', df['region_comb'])
df['region_comb'] = np.where(df['HH7A'] == 'Ho Chi Minh city', 'Ho Chi Minh city', df['region_comb'])

# --- Mother's education --- #

## Create categories
conditions = [
    ((df['WB6A'].isnull()) | (df['WB6A'] == 'ECE')),
    (df['WB6A'] == 'PRIMARY'),
    ((df['WB6A'] == 'LOWER SECONDARY') | (df['WB6A'] == 'UPPER SECONDARY') |
     (df['WB6A'] == 'VOCATIONAL HIGH SCHOOL')),
    (df['WB6A'] == 'UNIVERSITY/ COLLEGE OR HIGHER')
]
choices = ['Edu HoH: None', 'Edu HoH: Primary', 'Edu HoH: Secondary', 'Edu HoH: Higher']
df['mother_edu'] = np.select(conditions, choices)

# --- Ethnicity of HoH --- #
df['ethn_hoh'] = df['ethnicity']

# --- Female HoH --- #
df['sex_HoH'] = np.where(df['HL4'] == 'FEMALE', 'Sex HoH: Female', 'Sex HoH: Male')

# --- Elderly HoH --- #
### Convert 95+
df['HL6'] = np.where(df['HL6'] == '95+', 'NaN', df['HL6'])

### Cast value to float 
df['HL6'] = df['HL6'].astype(float)

df['elderly_HoH'] = np.where(df['HL6'] >= 60, 'Elderly HoH: YES', 'Elderly HoH: NO')

#### Subset and export working dataset

In [ ]:
# Identify and select columns for working dataset
working_var_idx = df.columns.get_loc('Total')
working_var_cols = df.columns[working_var_idx:-1].to_list()

# Add weight variable
working_var_cols = working_var_cols + ['wmweight']

# Subeset the dataframe
out_df = df[working_var_cols]

# Generate out_filepath
out_file = recode.split('.')[0] + '_subset' + '.csv'
out_filepath = datadir.joinpath('clean').joinpath(out_file)

# Save as csv
out_df.to_csv(out_filepath, index=False)

### ---- 2014 ----

In [ ]:
# Identify data file of interest
recode = 'wm_2014.sav'

data_filepath = datadir.joinpath(recode)

print(data_filepath)

In [ ]:
# Read in data
df = pd.read_spss(data_filepath)

df.shape

In [ ]:
print(f'Rows: {df.shape[0]}; Columns: {df.shape[1]}')

# Drop column if all NaN
df = df.dropna(axis='columns', how='all')
print('Cleaning columns with NaN...')

print(f'Updated -- Rows: {df.shape[0]}; Columns: {df.shape[1]}')

In [ ]:
# Check if any lines are duplicates
print('The are no duplicate rows: ' + str(sum(df.duplicated()) == 0))

# Find duplicate rows
print(f'There are {df.duplicated().sum()} duplicates in the dataset')

In [ ]:
# Select completed questionnaires (WM7) and birth in past two years (CM113)
print(df.shape)
df = df[(df['WM7'] == "Completed") & (df['CM13'] == "Yes")]
print(df.shape)

In [ ]:
# Add Total column
df['Total'] = 'Total'

# Add Year column
df['Year'] = '2014'

In [ ]:
# Merge in Sex HoH, Age HoH from HH recode

# zfill front of HH1, HH2 and concatenate to create unique HHID
df['HH1'] = df['HH1'].astype(str).str.strip().str.replace('.', '-').str.replace('-0', '').str.zfill(3)
df['HH2'] = df['HH2'].astype(str).str.strip().str.replace('.', '-').str.replace('-0', '').str.zfill(2)

df['HHID'] = df['HH1'] + df['HH2']

# Merge Age_HoH and Sex_HoH
hh_merge_df = pd.read_csv('./merge/hh_merge_2014.csv', dtype={'HHID': 'string'})
df = df[:].merge(hh_merge_df, on='HHID', how='left')

### Create Outcome Indicators

In [ ]:
# --- ANC 4+ visits [anc_4_visits] --- #

## Convert categorical column to int

### Convert DK and NaN to 99
df['MN3'] = np.where(df['MN3'] == 'DK', '99.0', df['MN3'])

### Cast value to float 
df['MN3'] = df['MN3'].astype(float)

## Create categories
conditions = [
    (df['MN3'] == 0),
    ((df['MN3'] >= 1) & (df['MN3'] <= 3)),
    (df['MN3'] >= 4) &(df['MN3'] < 99),
    (df['MN3'] >= 99)
]
choices = ['None', '1-3 visits', '4+ visits', 'DK/Missing']
df['anc_no_visits'] = np.select(conditions, choices)

# Create indicator
df['anc_4_visits'] = np.where(df.anc_no_visits == '4+ visits', 100, 0)

In [ ]:
# --- ANC components [anc_3_components] --- #

# Create indicator
df['anc_3_components'] = np.where((df.MN4A == 'Yes') & (df.MN4B == 'Yes') & (df.MN4C == 'Yes'), 100, 0)

In [ ]:
# --- Institutional [inst_delivery] --- #

## Create categories
inst_delivery_options = ['Government hospital', 'Private hospital', 'Commune health centre', 
'Private clinic', 'Policlinic', 'Other public', 'Private maternity home']

# Create indicator
df['inst_delivery'] = np.where(df['MN18'].isin(inst_delivery_options), 100, 0)

In [ ]:
# --- Caesarean Delivery [caesarean_del] --- #

# Create indicator
df['caesarean_del'] = np.where(df['MN19'] == 'Yes', 100, 0)

In [ ]:
# --- Post-natal Health Check (mother) [pnc_mother] --- #

## Health check by health provider after birth & a) before leaving facility (PN4) or b) before health provider left home (PN8)

## Create sub-indicator
df['health_check_after_birth'] = np.where(((df['PN4'] == 'Yes') | (df['PN8'] == 'Yes')), 100, 0)

## Seprate PNC visit by health provider (PN21U and PN21N)

## Post-natal care visit within 2 days ##

### Convert PN21N to float
df['PN21N'] = np.where(df['PN21N'] == 'DK', '99.0', df['PN21N'])
df['PN21N'] = df['PN21N'].astype(float)

## Create sub-indicator
df['pnc_2_days'] = np.where((df['PN21U'] == 'Hours') | ((df['PN21U'] == 'Days') & (df['PN21N'] <= 2)), 100, 0)

### Set value pnc_2_days to 0 IF check was not done by health provider
df['pnc_health_provider'] = np.where((df['PN22A'] == 'Doctor') | (df['PN22B'] == 'Nurse / Midwife') |\
     (df['PN22F'] == 'Traditional birth attendant') | (df['PN22G'] == 'Village health worker'), 100, 0)

df['pnc_2_days'] = np.where((df['pnc_health_provider'] == 100), df['pnc_2_days'], 0)

# Create indicator
df['pnc_mother'] = np.where((df['health_check_after_birth'] == 100) | (df['pnc_2_days'] == 100), 100, 0)

In [ ]:
########### NEED TO FIX !!!!! CHECK CITED REFERENCE IN 2014 REPORT ##########

# --- Low birthweight [low_bw] --- #

## Convert categorical column to int

### Convert DK and NaN to 99
df['MN22'] = np.where(df['MN22'] == 'DK', 'NaN', df['MN22'])

### Cast value to float 
df['MN22'] = df['MN22'].astype(float)

# Create indicator
df['low_bw'] = np.where(df['MN22'] < 2.5, 100, 0)
df['low_bw'] = np.where(df['MN22'].isnull(), np.nan, df['low_bw'])

In [ ]:
# --- Early Initiation BF [early_bf] --- #
### Convert MN25N to float
df['MN25N'] = np.where(df['MN25N'] == 'DK', '999.0', df['MN25N'])
df['MN25N'] = df['MN25N'].astype(float)

# Create indicator
df['early_bf'] = np.where((df['MN25U'] == 'Immediately') | ((df['MN25U'] == 'Hours') & (df['MN25N'] < 1)), 100, 0)

### Create Equity Stratifiers

In [ ]:
df.head(1)

In [ ]:
df.ethnicity.value_counts(dropna=False)

In [ ]:
# --- Residence --- #
df['residence'] = np.where(df['HH6'] == 'Rural', 'Rural', 'Urban')

# # --- Location --- #

# ## Combine region value with city value
# df['region_comb'] = df['HH7']
# df['region_comb'] = np.where(df['HH7A'] == 'Ha Noi', 'Ha Noi', df['region_comb'])
# df['region_comb'] = np.where(df['HH7A'] == 'Ho Chi Minh city', 'Ho Chi Minh city', df['region_comb'])

# --- Mother's education --- #

## Create categories
conditions = [
    ((df['WB4'].isnull()) | (df['WB4'] == 'Preschool')),
    (df['WB4'] == 'Primary'),
    ((df['WB4'] == 'Lower Secondary') | (df['WB4'] == 'Upper Secondary') |
     (df['WB4'] == 'Professional School')),
    (df['WB4'] == 'College/University & above')
]
choices = ['Edu HoH: None', 'Edu HoH: Primary', 'Edu HoH: Secondary', 'Edu HoH: Higher']
df['mother_edu'] = np.select(conditions, choices)

# --- Ethnicity of HoH --- #
df['ethn_hoh'] = df['ethnicity']

# --- Female HoH --- #
df['sex_HoH'] = np.where(df['HL4'] == 'FEMALE', 'Sex HoH: Female', 'Sex HoH: Male')

# --- Elderly HoH --- #
### Convert 95+
df['HL6'] = np.where(df['HL6'] == '95+', 'NaN', df['HL6'])

### Cast value to float 
df['HL6'] = df['HL6'].astype(float)

df['elderly_HoH'] = np.where(df['HL6'] >= 60, 'Elderly HoH: YES', 'Elderly HoH: NO')

In [ ]:
# Weighted mean
wt = ['wmweight']
var = 'early_bf'
ind_vars = ['HH7']

output_mean_table(df, var, ind_vars, wt)